In [32]:
import tensorflow as tf
print("Tensorflow: v" + tf.__version__)

Tensorflow: v2.4.1


## [Weight Clustering in Keras](https://www.tensorflow.org/model_optimization/guide/clustering/clustering_example)

Clustering reduces the number of unique weight values in a model, leading to benefits for deployment. It first groups the weights of each layer into N clusters, then shares the cluster's centroid value for all the weights belonging to the cluster.

In [33]:
pip install -q tensorflow-model-optimization

Note: you may need to restart the kernel to use updated packages.


In [34]:
from tensorflow import keras

import numpy as np
import tempfile
import zipfile
import os

### Train a tf.keras model for MNIST without clustering

In [35]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [36]:
# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images  = test_images / 255.0

In [37]:
# Define the model architecture.
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28, 28)),
    keras.layers.Reshape(target_shape=(28, 28, 1)),
    keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation=tf.nn.relu),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(10)
])


In [38]:
# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_images, train_labels, validation_split=0.1, epochs=10)

Epoch 1/10
1688/1688 [==============================] - 4s 2ms/step - loss: 0.5263 - accuracy: 0.8562 - val_loss: 0.1018 - val_accuracy: 0.9740
Epoch 2/10
1688/1688 [==============================] - 3s 2ms/step - loss: 0.1104 - accuracy: 0.9684 - val_loss: 0.0765 - val_accuracy: 0.9793
Epoch 3/10
1688/1688 [==============================] - 3s 2ms/step - loss: 0.0751 - accuracy: 0.9775 - val_loss: 0.0668 - val_accuracy: 0.9822
Epoch 4/10
1688/1688 [==============================] - 3s 2ms/step - loss: 0.0614 - accuracy: 0.9821 - val_loss: 0.0672 - val_accuracy: 0.9822
Epoch 5/10
1688/1688 [==============================] - 3s 2ms/step - loss: 0.0528 - accuracy: 0.9843 - val_loss: 0.0664 - val_accuracy: 0.9828
Epoch 6/10
1688/1688 [==============================] - 3s 2ms/step - loss: 0.0515 - accuracy: 0.9850 - val_loss: 0.0607 - val_accuracy: 0.9838
Epoch 7/10
1688/1688 [==============================] - 3s 2ms/step - loss: 0.0437 - accuracy: 0.9871 - val_loss: 0.0600 - val_accuracy:

### Evaluate the baseline model 

In [39]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file)
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

Baseline test accuracy: 0.982699990272522
Saving model to:  /tmp/tmpne8p2j60.h5


### Fine-tune the pre-trained model with clustering

In [40]:
import tensorflow_model_optimization as tfmot

cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = { 
    'number_of_clusters': 16,
    'cluster_centroids_init': CentroidInitialization.LINEAR
}

# Cluster a whole model
clustered_model = cluster_weights(model, **clustering_params)

# Use smaller learning rate for fine-tuning clustered model
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

clustered_model.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=opt,
  metrics=['accuracy'])

clustered_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cluster_reshape_5 (ClusterWe (None, 28, 28, 1)         0         
_________________________________________________________________
cluster_conv2d_3 (ClusterWei (None, 26, 26, 12)        136       
_________________________________________________________________
cluster_max_pooling2d_3 (Clu (None, 13, 13, 12)        0         
_________________________________________________________________
cluster_flatten_3 (ClusterWe (None, 2028)              0         
_________________________________________________________________
cluster_dense_3 (ClusterWeig (None, 10)                20306     
Total params: 20,442
Trainable params: 54
Non-trainable params: 20,388
_________________________________________________________________


### Comparing against the baseline

In [50]:
# Fine-tune model
clustered_model.fit(
  train_images,
  train_labels,
  batch_size=500,
  epochs=1,
  validation_split=0.1)

108/108 [==============================] - 1s 7ms/step - loss: 0.0408 - accuracy: 0.9868 - val_loss: 0.0745 - val_accuracy: 0.9803


In [51]:
_, clustered_model_accuracy = clustered_model.evaluate(test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Clustered test accuracy:', clustered_model_accuracy)

Baseline test accuracy: 0.982699990272522
Clustered test accuracy: 0.9785000085830688


### Create 6x smaller models from clustering

In [43]:
final_model = tfmot.clustering.keras.strip_clustering(clustered_model)

_, clustered_keras_file = tempfile.mkstemp('.h5')
print('Saving clustered model to: ', clustered_keras_file)
tf.keras.models.save_model(final_model, clustered_keras_file, 
                           include_optimizer=False)

Saving clustered model to:  /tmp/tmpjn4bl2kr.h5


In [44]:
clustered_tflite_file = '/tmp/clustered_mnist.tflite'
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
tflite_clustered_model = converter.convert()
with open(clustered_tflite_file, 'wb') as f:
  f.write(tflite_clustered_model)
print('Saved clustered TFLite model to:', clustered_tflite_file)

INFO:tensorflow:Assets written to: /tmp/tmpwbntkryx/assets


INFO:tensorflow:Assets written to: /tmp/tmpwbntkryx/assets


Saved clustered TFLite model to: /tmp/clustered_mnist.tflite


In [45]:
def get_gzipped_model_size(file):
  # It returns the size of the gzipped model in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [52]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped clustered Keras model: %.2f bytes" % (get_gzipped_model_size(clustered_keras_file)))
print("Size of gzipped clustered TFlite model: %.2f bytes" % (get_gzipped_model_size(clustered_tflite_file)))

Size of gzipped baseline Keras model: 78153.00 bytes
Size of gzipped clustered Keras model: 13338.00 bytes
Size of gzipped clustered TFlite model: 12686.00 bytes


### Create an 8x smaller TFLite model from combining weight clustering and post-training quantization

In [53]:
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

_, quantized_and_clustered_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_clustered_tflite_file, 'wb') as f:
  f.write(tflite_quant_model)

print('Saved quantized and clustered TFLite model to:', quantized_and_clustered_tflite_file)
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped clustered and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_clustered_tflite_file)))

INFO:tensorflow:Assets written to: /tmp/tmpbxb0rrd5/assets


INFO:tensorflow:Assets written to: /tmp/tmpbxb0rrd5/assets


Saved quantized and clustered TFLite model to: /tmp/tmp748o3_r0.tflite
Size of gzipped baseline Keras model: 78153.00 bytes
Size of gzipped clustered and quantized TFlite model: 9592.00 bytes


### See the persistence of accuracy from TF to TFLite

In [54]:
def eval_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

In [55]:
interpreter = tf.lite.Interpreter(model_content=tflite_quant_model)
interpreter.allocate_tensors()

test_accuracy = eval_model(interpreter)

print('Clustered and quantized TFLite test_accuracy:', test_accuracy)
print('Clustered TF test accuracy:', clustered_model_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.


Clustered and quantized TFLite test_accuracy: 0.9787
Clustered TF test accuracy: 0.9785000085830688
